#### Problem:

Write a function called `included_sets(s)` which includes all of the sets contained within a given set `s`.

For instance, given the set `s = {4, 1}`, the output would include the sets `{}, {4}, {1}, {4, 1}`.

#### Thoughts

This problem screams "use recursion". To do that, we could have `included_sets()` add the given set to a list, and then interatively call itself on the same set minus one of each of the elements. Append the results of each iteration to the final list. For instance:

In [48]:
def included_sets(s):
    res = [s]
    for e in list(s):
        s0 = s.copy()
        s0.remove(e)
        sets = included_sets(s0)
        res = res + sets
    return res

included_sets({4, 1, 9})

[{1, 4, 9},
 {4, 9},
 {9},
 set(),
 {4},
 set(),
 {1, 9},
 {9},
 set(),
 {1},
 set(),
 {1, 4},
 {4},
 set(),
 {1},
 set()]

The output does indeed include all the sets that it needs to include, but there are a couple problems:

* The output contains duplicates.
* Functional recursion has too much overhead.

Let's solve these problems separately.

#### Avoiding duplicates

One way to avoid the duplicates is to filter them out of the `res` list at the end of the function. However, the filtering would then happen with each individual call to `included_sets()`. A better idea would be to wrap the first call to the recursive function with a different function that does the filtering at the end, so the filtering only happens one time total. Like so:

In [72]:
def _included_sets(s):
    res = [s]
    for e in list(s):
        s0 = s.copy()
        s0.remove(e)
        sets = included_sets(s0)
        res = res + sets
    return res

def included_sets(s):
    sets = _included_sets(s)
    res = []
    for s0 in sets:
        if s0 not in res:
            res.append(s0)
    return res

included_sets({4, 1, 9})

[{1, 4, 9}, {4, 9}, {9}, set(), {4}, {1, 9}, {1}, {1, 4}]

There is also a slightly simpler idea, which is to only use the one recursive function, and to filter out duplicates before adding them to `res` in the first place. Like so:

In [71]:
def included_sets(s):
    res = [s]
    for e in list(s):
        s0 = s.copy()
        s0.remove(e)
        sets = included_sets(s0)
        for s1 in sets:
            if s1 not in res:
                res.append(s1)
    return res

included_sets({4, 1, 9})

[{1, 4, 9}, {4, 9}, {9}, set(), {4}, {1, 9}, {1}, {1, 4}]

This solution is more memory efficient than the previous one, since it avoids creating any lists that are larger than the final output.

However, the performance of both of these solutions is still significantly worse than we can make it. Can we prune some or all of the unnecessary recursive calls before they ever happen, rather than filter their results out later?

Consider the call graph for `included_sets(s)`. Each node of the graph has the corresponding value of the argument `s`.

Remember that `included_sets()` will execute these calls in depth-first order.

Look at each of the three subgraphs branching off from `{1, 4, 9}`--specifially, the subgraphs at root nodes `{4, 9}`, `{1, 9}`, and `{1, 4}`. Each of these subgraphs is three levels deep, and they contain the same number of nodes.

Observe that the second subgraph's third level can only contain sets which were included in the first subgraph, the set `{}`. This means that we do not even need to visit the third level of the second subgraph.

Similarly, the third subgraph's second and third levels only contain sets which were included in the first two subgraphs, the sets `{}`, `{1}`, and `{4}`. This means that we do not need to visit the second and third level of the third subgraph.

We can prune these branches by keeping a `max_depth` variable which gets decremented for each iteration of the outer for-loop of `included_sets()`. `max_depth` will be passed to each recursive call to prevent traversing any deeper after `max_depth` reaches 0.

In [82]:
def _included_sets(s, *, max_depth):
    res = [s]
    for e in list(s):
        max_depth -= 1
        if max_depth < 0:
            break
        s0 = s.copy()
        s0.remove(e)
        sets = _included_sets(s0, max_depth=max_depth)
        res = res + sets
    return res

def included_sets(s):
    return _included_sets(s, max_depth=len(s))

included_sets({4, 1, 9})

[{1, 4, 9}, {4, 9}, {9}, set(), {4}, {1, 9}, {9}, {1, 4}]

Now we're avoiding the duplicates without ever having to iterate through `res` to filter out duplicates.